In [1]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [3]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [4]:
from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import _fetch_batch, fetch_data, fetch_spot_data

In [5]:
from expiries import dict_expiries

In [6]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("o").first().alias("o"),
                pl.col("h").max().alias("h"),
                pl.col("l").min().alias("l"),
                pl.col("c").last().alias("c"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [7]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2019, 2026),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe

    # Iterate over each year
    for year in range(2019, 2026):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()

        # Calculate total number of trades
        total_trades = len(year_trades)

        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100

        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()

        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()

        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()

        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = f"{variation}"

        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]

    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0][
        "ROI%"
    ].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum()
        - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio: stats_df8}

In [8]:
bnf_1min = pd.read_csv("../data/nifty_min.csv")

In [9]:
bnf_1min.columns = ['index', 'datetime', 'o', 'h', 'l', 'c', 'v']
bnf_1min.tail()

,index,datetime,o,h,l,c,v
750760,nifty,2025-03-28 15:25:00,23500.2500,23502.5000,23485.7500,23490.4000,0
750761,nifty,2025-03-28 15:26:00,23490.7500,23494.3500,23486.7500,23494.0500,0
750762,nifty,2025-03-28 15:27:00,23494.2000,23497.4500,23489.7500,23496.8000,0
750763,nifty,2025-03-28 15:28:00,23497.1000,23500.4500,23491.0500,23492.0000,0
750764,nifty,2025-03-28 15:29:00,23492.2500,23503.1000,23450.2000,23495.1500,0


In [10]:
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"]).dt.tz_localize(None)
# bnf_1min = bnf_1min[((bnf_1min['datetime'].dt.year == 2020) & (bnf_1min['datetime'].dt.month == 4))]
bnf_1min = bnf_1min[
    (bnf_1min["datetime"].dt.year >= 2019) & (bnf_1min["datetime"].dt.year <= 2025)
]

In [11]:
dict_expiries

{'nifty': [datetime.datetime(2017, 1, 25, 0, 0),
  datetime.datetime(2017, 2, 23, 0, 0),
  datetime.datetime(2017, 3, 30, 0, 0),
  datetime.datetime(2017, 4, 27, 0, 0),
  datetime.datetime(2017, 5, 25, 0, 0),
  datetime.datetime(2017, 6, 29, 0, 0),
  datetime.datetime(2017, 7, 27, 0, 0),
  datetime.datetime(2017, 8, 31, 0, 0),
  datetime.datetime(2017, 9, 28, 0, 0),
  datetime.datetime(2017, 10, 26, 0, 0),
  datetime.datetime(2017, 11, 30, 0, 0),
  datetime.datetime(2017, 12, 28, 0, 0),
  datetime.datetime(2018, 1, 25, 0, 0),
  datetime.datetime(2018, 2, 22, 0, 0),
  datetime.datetime(2018, 3, 28, 0, 0),
  datetime.datetime(2018, 4, 26, 0, 0),
  datetime.datetime(2018, 5, 31, 0, 0),
  datetime.datetime(2018, 6, 28, 0, 0),
  datetime.datetime(2018, 7, 26, 0, 0),
  datetime.datetime(2018, 8, 30, 0, 0),
  datetime.datetime(2018, 9, 27, 0, 0),
  datetime.datetime(2018, 10, 25, 0, 0),
  datetime.datetime(2018, 11, 29, 0, 0),
  datetime.datetime(2018, 12, 27, 0, 0),
  datetime.datetime(2019,

In [12]:
from datetime import date
from bisect import bisect_left

def get_next_expiry(input_date, index_symbol):
    expiries = dict_expiries.get(index_symbol)
    if not expiries:
        return None
        
    expiry_dates = sorted({dt.date() for dt in expiries})
    pos = bisect_left(expiry_dates, input_date)
    return expiry_dates[pos] if pos < len(expiry_dates) else None


In [13]:
index_ = 'nifty'

if index_ == 'nifty':
    LOT_SIZE_ = 75
    STRIKE_SPREAD_ = 50
    INDEX_LEVERAGE_ = 8
    PORTFOLIO_ = 1_00_00_000

In [14]:
import pandas as pd

async def add_atr(df, period=14):
    """
    Adds an 'ATR' column to the DataFrame using Wilder's smoothing (like TradingView).
    
    Parameters:
    df (pd.DataFrame): Must contain 'h', 'l', 'c' columns for high, low, close
    period (int): ATR period (default 14)
    
    Returns:
    pd.DataFrame: With 'ATR' column added
    """
    high = df['h']
    low = df['l']
    close = df['c']

    # True Range
    tr1 = high - low
    tr2 = (high - close.shift()).abs()
    tr3 = (low - close.shift()).abs()
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)

    # ATR with Wilder's smoothing (like an EMA with alpha=1/period)
    atr = tr.ewm(alpha=1/period, adjust=False).mean()

    df['ATR'] = atr
    return df


In [15]:
async def send_trade(spot_price, entry_timestamp, expiry, direction, atr, tf, offset, dte, tradebook, trade_num):
    if direction == 1:
        opt_type = 'P'
        side = 'LONG'
    elif direction == -1:
        opt_type = "C"
        side = 'SHORT'
    else:
        return tradebook

    current_date = entry_timestamp.date()
    current_time = entry_timestamp.time()

    strike = int(round(spot_price / STRIKE_SPREAD_) * STRIKE_SPREAD_)

    option_df = await fetch_data(
        index=index_,
        expiry=expiry,
        strike=strike,
        asset_class=opt_type,
        start_date=current_date,
        start_time=entry_timestamp.time(),
        end_date=current_date,
        end_time=dt.time(15, 30),
    )

    if option_df is not None and not isinstance(option_df, str):
        option_df = resample(option_df, tf, offset)
        option_df_pandas = option_df.to_pandas()
        entry_price = option_df_pandas.iloc[0]['c']
    else:
        entry_price = float('nan')

    # print(f'{entry_timestamp} : {side} TRADE Entered ! ')

    trade = {
        'date': current_date,
        # 'high level': high_level,
        # 'low level': low_level,
        # 'atr_multiplier': multiplier,
        'Morning ATR': atr,
        'side': side,
        'strike': strike,
        'type': opt_type,
        'expiry': expiry,
        'dte': dte,
        'entry_time': current_time,
        'entry price': entry_price,
        'trade_num': trade_num,
    }
        
    tradebook.append(trade)
    return tradebook

async def take_exit(strike, opt_type, expiry, exit_timestamp, tf, offset, tradebook, trade_num):

    current_date = exit_timestamp.date()
    current_time = exit_timestamp.time()

    strike = int(round(strike / STRIKE_SPREAD_) * STRIKE_SPREAD_)

    if opt_type == 'P':
        side = 'LONG'
    elif opt_type == 'C':
        side = 'SHORT'
    else:
        side = 'None'

    for trade in tradebook:
        if (
            trade['type'] == opt_type and
            trade['date'] == current_date and
            'exit_time' not in trade
        ):
            option_df = await fetch_data(
                index=index_,
                expiry=expiry,
                strike=strike,
                asset_class=opt_type,
                start_date=current_date,
                start_time=current_time,
                end_date=current_date,
                end_time=dt.time(15, 30),
            )
            # print(option_df)
            if option_df is not None and not isinstance(option_df, str):
                option_df = resample(option_df, tf, offset)
                option_df_pandas = option_df.to_pandas()
                exit_price = option_df_pandas.iloc[0]['c']
                # print(f'{side} Trade Exited')
            else:
                exit_price = float('nan')

            if current_time > dt.time(15, 20):
                remark = 'EOD Exit'
            else:
                remark = 'TSL Hit'

            # print(f'{exit_timestamp} : {side} {remark}')

            if option_df is not None and not isinstance(option_df, str):
                trade['exit price'] = exit_price
                trade['exit_time'] = option_df_pandas['datetime'].iloc[0] if len(option_df_pandas) != 0 else float('nan')
                trade['remarks'] = remark
                # trade['high level at exit'] = high_level
                # trade['low level at exit'] = low_level
                trade['points'] = trade['entry price'] - trade['exit price']
            else:
                trade['exit price'] = float('nan')
                trade['exit_time'] = float('nan')
                trade['remarks'] = remark
                # trade['high level at exit'] = high_level
                # trade['low level at exit'] = low_level
                trade['points'] = trade['entry price'] - trade['exit price']

    return tradebook

In [19]:
async def backtest_intraday_levels2(df, multiplier, tf, offset):
    df['datetime'] = pd.to_datetime(df['datetime'])
    # df = df[df['datetime'].dt.year >= 2025]
    df['date'] = df['datetime'].dt.date
    # print(df.head().to_string())
    
    tradebook = []

    eod_time = dt.time(15, 20)
    no_more_trade_time = dt.time(14, 0)

    for date, group in df.groupby('date'):
        current_date = date
        group = group.reset_index(drop=True)
        # print(date)
        # Get 9:15 candle
        morning_candle = group[group['datetime'].dt.time == pd.to_datetime("09:15").time()]
        if morning_candle.empty:
            continue

        morning_atr = morning_candle.iloc[0]['ATR']
        running_high = group.iloc[0]['h'] # Initialising
        running_low = group.iloc[0]['l'] # Initialising

        is_high_breached = False
        is_low_breached = False

        in_trade_long = False
        in_trade_short = False

        data_fetched_pe = False
        data_fetched_ce = False

        trade_num = 1
        max_trades = 25

        for i in range(0, len(group)):
            row = group.iloc[i]
            previous_row = group.iloc[i-1]
            running_high = max(running_high, row['h'])
            running_low = min(running_low, row['l'])
            
            current_datetime = row['datetime']
            
            high_level = running_low + (multiplier * morning_atr)
            low_level = running_high - (multiplier * morning_atr)
            
            long_tsl = low_level
            short_tsl = high_level
            
            # print(row)
            # print(group.iloc[i]['datetime'], high_level, low_level, row['c'])
            # print(f'RH : {running_high} , RL : {running_low}')

            # high_strike = int(round(high_level / STRIKE_SPREAD_) * STRIKE_SPREAD_)
            # low_strike = int(round(low_level / STRIKE_SPREAD_) * STRIKE_SPREAD_)

            expiry = get_next_expiry(date, index_)
            dte = (expiry - current_date).days            

            if not in_trade_long and not in_trade_short and current_datetime.time() < no_more_trade_time:
                if row['c'] >= high_level and trade_num <= max_trades:
                    tradebook = await send_trade(high_level, current_datetime, expiry, 1, morning_atr, tf, offset, dte, tradebook, trade_num)
                    trade_num += 1
                    # is_high_breached = True
                    in_trade_long = True
                    long_tsl = low_level

            if in_trade_long:
                if row['c'] <= long_tsl:
                    # TSL Hit
                    tradebook = await take_exit(high_level, 'P', expiry, current_datetime, tf, offset, tradebook, trade_num)
                    in_trade_long = False
                    # is_high_breached = False

                    # Send Short Trade
                    running_low = row['l']
                    # print(f'Running Low changed to {running_low}')
                    high_level = running_low + (multiplier * morning_atr)
                    # print(f'High Level changed to {high_level}')
                    if trade_num <= max_trades and current_datetime.time() <= no_more_trade_time:
                        tradebook = await send_trade(long_tsl, current_datetime, expiry, -1, morning_atr, tf, offset, dte, tradebook, trade_num)
                        trade_num += 1
                        long_tsl = float('inf')
                        short_tsl = high_level
                        in_trade_short = True
                        # is_low_breached = True
                    else:
                        continue
                    
                elif current_datetime.time() >= eod_time:
                    # EOD Exit
                    tradebook = await take_exit(high_level, 'P', expiry, current_datetime, tf, offset, tradebook, trade_num)
    
                    in_trade_long = False
                    # is_high_breached = False
                    long_tsl = float('inf')

            if not in_trade_short and not in_trade_long and current_datetime.time() < no_more_trade_time:
                if row['c'] <= low_level and trade_num <= max_trades:
                    tradebook = await send_trade(low_level, current_datetime, expiry, -1, morning_atr, tf, offset, dte, tradebook, trade_num)
                    trade_num += 1
                    
                    # is_low_breached = True
                    in_trade_short = True

            if in_trade_short:
                if row['c'] >= short_tsl:
                    # TSL Hit
                    tradebook = await take_exit(low_level, 'C', expiry, current_datetime, tf, offset, tradebook, trade_num)
                    
                    in_trade_short = False
                    # is_low_breached = False

                    # Send Long Trade
                    running_high = row['h']
                    # print(f'Running High changed to {running_high}')
                    low_level = running_high - (multiplier * morning_atr)
                    # print(f'Low Level changed to {low_level}')
                    if trade_num <= max_trades and current_datetime.time() <= no_more_trade_time:
                        tradebook = await send_trade(short_tsl, current_datetime, expiry, 1, morning_atr, tf, offset, dte, tradebook, trade_num)
                        trade_num += 1
                        short_tsl = float('-inf')
                        long_tsl = low_level
                        in_trade_long = True
                        # is_high_breached = True
                    else:
                        continue

                elif current_datetime.time() >= eod_time:
                    # EOD Exit
                    tradebook = await take_exit(low_level, 'C', expiry, current_datetime, tf, offset, tradebook, trade_num)

                    in_trade_short = False
                    # is_low_breached = False
                    short_tsl = float('-inf')

    tb = pd.DataFrame(tradebook)
    if len(tb) > 0:
        tb['exit_time'] = pd.to_datetime(tb['exit_time'])
        tb['slippage'] = 0.01 * (tb['entry price'] + tb['exit price'])
        tb['final_points'] = tb['points'] - tb['slippage']
        tb['portfolio'] = PORTFOLIO_
        tb['index leverage'] = np.where(tb['type'] == 'C', 8, 8)
        tb['qty'] = tb['portfolio'] * tb['index leverage'] / tb['strike']
        tb['pnl'] = tb['final_points'] * tb['qty']
        tb['ROI%'] = tb['pnl'] * 100 / tb['portfolio']
        tb['Trade Year'] = tb['exit_time'].dt.year
        # tb = tb[tb['entry_time'] != tb['exit_time']]

    return tb

In [17]:
TF_ = '10m'
OFFSET_ = '5m'
ATR_WINDOW_ = 14
MULTIPLIER_ = 2
bnf_df = resample(pl.DataFrame(bnf_1min), TF_, OFFSET_)
bnf_df_pandas = bnf_df.to_pandas()
bnf_df_pandas = await add_atr(bnf_df_pandas, ATR_WINDOW_)
tb = await backtest_intraday_levels2(bnf_df_pandas, MULTIPLIER_, TF_, OFFSET_)
# tb

CancelledError: 

In [19]:
tb[tb['Trade Year'] >= 2025]

,date,Morning ATR,side,strike,type,expiry,dte,entry_time,entry price,trade_num,exit price,exit_time,remarks,points,slippage,final_points,portfolio,index leverage,qty,pnl,ROI%,Trade Year
3905,2025-01-01,35.1421,SHORT,23600,C,2025-01-02,1,09:35:00,137.2500,1,169.9500,2025-01-01 10:25:00,TSL Hit,-32.7000,3.0720,-35.7720,10000000,8,3389.8305,-121261.0169,-1.2126,2025.0000
3906,2025-01-01,35.1421,LONG,23650,P,2025-01-02,1,10:25:00,108.9500,2,105.7000,2025-01-01 11:45:00,TSL Hit,3.2500,2.1465,1.1035,10000000,8,3382.6638,3732.7696,0.0373,2025.0000
3907,2025-01-01,35.1421,SHORT,23700,C,2025-01-02,1,11:45:00,146.6000,3,197.3000,2025-01-01 12:15:00,TSL Hit,-50.7000,3.4390,-54.1390,10000000,8,3375.5274,-182747.6793,-1.8275,2025.0000
3908,2025-01-01,35.1421,LONG,23750,P,2025-01-02,1,12:15:00,117.2000,4,106.5500,2025-01-01 14:45:00,TSL Hit,10.6500,2.2375,8.4125,10000000,8,3368.4211,28336.8421,0.2834,2025.0000
3909,2025-01-01,35.1421,SHORT,23750,C,2025-01-02,1,14:45:00,129.1500,5,146.0000,2025-01-01 15:25:00,EOD Exit,-16.8500,2.7515,-19.6015,10000000,8,3368.4211,-66026.1053,-0.6603,2025.0000
3910,2025-01-02,30.9336,LONG,23800,P,2025-01-02,0,09:35:00,85.1000,1,0.0500,2025-01-02 15:25:00,EOD Exit,85.0500,0.8515,84.1985,10000000,8,3361.3445,283020.1681,2.8302,2025.0000
3911,2025-01-03,35.4638,SHORT,24150,C,2025-01-09,6,09:25:00,158.9000,1,149.6000,2025-01-03 11:15:00,TSL Hit,9.3000,3.0850,6.2150,10000000,8,3312.6294,20587.9917,0.2059,2025.0000
3912,2025-01-03,35.4638,LONG,24050,P,2025-01-09,6,11:15:00,130.2500,2,153.2000,2025-01-03 12:15:00,TSL Hit,-22.9500,2.8345,-25.7845,10000000,8,3326.4033,-85769.6466,-0.8577,2025.0000
3913,2025-01-03,35.4638,SHORT,24100,C,2025-01-09,6,12:15:00,136.3500,3,105.8000,2025-01-03 15:25:00,EOD Exit,30.5500,2.4215,28.1285,10000000,8,3319.5021,93372.6141,0.9337,2025.0000
3914,2025-01-06,36.9781,SHORT,24000,C,2025-01-09,3,09:35:00,137.2500,1,157.0000,2025-01-06 09:55:00,TSL Hit,-19.7500,2.9425,-22.6925,10000000,8,3333.3333,-75641.6667,-0.7564,2025.0000


In [20]:
tb['ROI%'].sum()

727.47490839846

In [21]:
stats = generate_stats(tb, 'ATR Dynamic')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,114.3771,609,50.7389,1.1762,-0.8303,-9.1377,12.5170,ATR Dynamic
2020,251.0527,625,54.0800,1.9879,-1.4664,-23.4149,10.7219,ATR Dynamic
2021,94.2552,656,50.3049,1.2235,-0.9494,-24.0505,3.9191,ATR Dynamic
2022,134.4618,595,52.9412,1.3034,-0.9861,-20.9244,6.4261,ATR Dynamic
2023,21.3142,640,47.1875,0.8364,-0.6842,-18.4602,1.1546,ATR Dynamic
2024,80.8750,724,47.6519,1.0843,-0.7736,-24.8346,3.2565,ATR Dynamic
2025,31.1388,193,58.0311,0.9362,-0.9100,-9.0518,3.4401,ATR Dynamic
Overall,727.4749,4042,50.0366,1.2585,-0.9311,-24.8346,29.2928,ATR Dynamic


In [22]:
stats = generate_stats(tb, 'ATR Dynamic')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,114.3771,609,50.7389,1.1762,-0.8303,-9.1377,12.5170,ATR Dynamic
2020,251.0527,625,54.0800,1.9879,-1.4664,-23.4149,10.7219,ATR Dynamic
2021,94.2552,656,50.3049,1.2235,-0.9494,-24.0505,3.9191,ATR Dynamic
2022,134.4618,595,52.9412,1.3034,-0.9861,-20.9244,6.4261,ATR Dynamic
2023,21.3142,640,47.1875,0.8364,-0.6842,-18.4602,1.1546,ATR Dynamic
2024,80.8750,724,47.6519,1.0843,-0.7736,-24.8346,3.2565,ATR Dynamic
2025,31.1388,193,58.0311,0.9362,-0.9100,-9.0518,3.4401,ATR Dynamic
Overall,727.4749,4042,50.0366,1.2585,-0.9311,-24.8346,29.2928,ATR Dynamic


In [23]:
# tb.to_csv('ATR_Dynamic_JJMS_10m_14_2.csv', index=False)

In [24]:
# tb = tb_with_hedge
tb['Cumulative ROI%'] = tb['ROI%'].cumsum()
tb['Max Cumulative ROI%'] = tb['Cumulative ROI%'].cummax()  # Maximum value so far
tb['DD'] = tb['Cumulative ROI%'] - tb['Max Cumulative ROI%']  # Drawdown
tb.tail()

,date,Morning ATR,side,strike,type,expiry,dte,entry_time,entry price,trade_num,exit price,exit_time,remarks,points,slippage,final_points,portfolio,index leverage,qty,pnl,ROI%,Trade Year,Cumulative ROI%,Max Cumulative ROI%,DD
4094,2025-03-28,29.4007,SHORT,23550,C,2025-04-03,6,09:15:00,189.6500,1,227.3000,2025-03-28 10:25:00,TSL Hit,-37.6500,4.1695,-41.8195,10000000,8,3397.0276,-142061.9958,-1.4206,2025.0000,725.5595,726.9801,-1.4206
4095,2025-03-28,29.4007,LONG,23550,P,2025-04-03,6,10:25:00,93.9500,2,109.0000,2025-03-28 11:05:00,TSL Hit,-15.0500,2.0295,-17.0795,10000000,8,3397.0276,-58019.5329,-0.5802,2025.0000,724.9793,726.9801,-2.0008
4096,2025-03-28,29.4007,SHORT,23600,C,2025-04-03,6,11:05:00,156.5000,3,93.0000,2025-03-28 14:15:00,TSL Hit,63.5000,2.4950,61.0050,10000000,8,3389.8305,206796.6102,2.0680,2025.0000,727.0473,727.0473,0.0000
4097,2025-03-28,29.4007,LONG,23500,P,2025-04-03,6,14:15:00,111.0000,4,98.1000,2025-03-28 14:55:00,TSL Hit,12.9000,2.0910,10.8090,10000000,8,3404.2553,36796.5957,0.3680,2025.0000,727.4152,727.4152,0.0000
4098,2025-03-28,29.4007,SHORT,23550,C,2025-04-03,6,14:55:00,116.7000,5,112.6500,2025-03-28 15:25:00,EOD Exit,4.0500,2.2935,1.7565,10000000,8,3397.0276,5966.8790,0.0597,2025.0000,727.4749,727.4749,0.0000


In [25]:
import pandas as pd

def calculate_dte_stats(df):

    # print(df.columns)
    if 'dte' not in df.columns or 'ROI%' not in df.columns or 'DD' not in df.columns:
        raise ValueError("DataFrame must contain 'DTE', 'returns', and 'drawdown' columns.")
    
    # Group by DTE and calculate required stats
    grouped_stats = df.groupby('dte').agg(
        returns_sum=('ROI%', 'sum'),
        max_drawdown=('DD', 'min'),  # Assuming 'drawdown' contains negative values
        total_trades=('dte', 'count')
    ).reset_index()
    
    # Calculate ratio of returns sum to max drawdown
    grouped_stats['returns_to_max_dd_ratio'] = (
        grouped_stats['returns_sum'] / grouped_stats['max_drawdown'].abs()
    )

    return grouped_stats


In [26]:
import pandas as pd

def calculate_opt_type_stats(df):

    # print(df.columns)
    if 'type' not in df.columns or 'ROI%' not in df.columns or 'DD' not in df.columns:
        raise ValueError("DataFrame must contain 'DTE', 'returns', and 'drawdown' columns.")
    
    # Group by DTE and calculate required stats
    grouped_stats = df.groupby('type').agg(
        returns_sum=('ROI%', 'sum'),
        max_drawdown=('DD', 'min'),  # Assuming 'drawdown' contains negative values
        total_trades=('type', 'count')
    ).reset_index()
    
    # Calculate ratio of returns sum to max drawdown
    grouped_stats['returns_to_max_dd_ratio'] = (
        grouped_stats['returns_sum'] / grouped_stats['max_drawdown'].abs()
    )

    return grouped_stats


In [27]:
import pandas as pd

def calculate_trade_num_stats(df):

    # print(df.columns)
    if 'trade_num' not in df.columns or 'ROI%' not in df.columns or 'DD' not in df.columns:
        raise ValueError("DataFrame must contain 'DTE', 'returns', and 'drawdown' columns.")
    
    # Group by DTE and calculate required stats
    grouped_stats = df.groupby('trade_num').agg(
        returns_sum=('ROI%', 'sum'),
        max_drawdown=('DD', 'min'),  # Assuming 'drawdown' contains negative values
        total_trades=('trade_num', 'count')
    ).reset_index()
    
    # Calculate ratio of returns sum to max drawdown
    grouped_stats['returns_to_max_dd_ratio'] = (
        grouped_stats['returns_sum'] / grouped_stats['max_drawdown'].abs()
    )

    return grouped_stats


In [28]:
stats_dte = calculate_dte_stats(tb)
stats_dte

,dte,returns_sum,max_drawdown,total_trades,returns_to_max_dd_ratio
0,0,221.2153,-24.0773,831,9.1877
1,1,134.4954,-24.8346,829,5.4156
2,2,172.9882,-24.3747,825,7.0970
3,3,120.7567,-24.2252,673,4.9848
4,5,-1.6365,-13.5432,56,-0.1208
5,6,81.2986,-24.0505,827,3.3803
6,7,0.2466,-3.8501,8,0.0640
7,8,0.9229,-5.3265,4,0.1733
8,9,0.2689,-3.3174,4,0.0811
9,10,0.9790,-3.5864,2,0.2730


In [29]:
stats_opt_type = calculate_opt_type_stats(tb)
stats_opt_type

,type,returns_sum,max_drawdown,total_trades,returns_to_max_dd_ratio
0,C,279.0507,-24.7315,1993,11.2832
1,P,448.4243,-24.8346,2106,18.0564


In [30]:
stats_trade_num = calculate_trade_num_stats(tb)
stats_trade_num

,trade_num,returns_sum,max_drawdown,total_trades,returns_to_max_dd_ratio
0,1,313.9153,-24.3747,1505,12.8787
1,2,177.4682,-24.1137,1155,7.3596
2,3,152.2019,-24.8346,755,6.1286
3,4,20.3934,-24.7315,391,0.8246
4,5,41.4301,-23.2589,156,1.7813
5,6,3.6560,-24.1415,68,0.1514
6,7,9.4540,-24.1267,31,0.3918
7,8,9.2791,-17.3485,21,0.5349
8,9,-1.7207,-8.6490,9,-0.1989
9,10,-0.2452,-6.0677,5,-0.0404


In [18]:
# SIMULATION

TF_ = ['10m']
ATR_WINDOW_ = [11, 12, 13, 14, 15, 16, 17]
MULTIPLIER_ = [1.5, 1.75, 2, 2.25, 2.5]

for i in TF_:
    for j in ATR_WINDOW_:
        for k in MULTIPLIER_:
            if i == '10m':
                z = '5m'
            else:
                z = '0m'
            variation = f'TF: {i}, ATR: {j}, MULT: {k}'
            print(variation)
            bnf_df = resample(pl.DataFrame(bnf_1min), i, z)
            bnf_df_pandas = bnf_df.to_pandas()
            bnf_df_pandas = await add_atr(bnf_df_pandas, j)
            tb = await backtest_intraday_levels2(bnf_df_pandas, k, i, z)

            if len(tb) > 0:
                stats = generate_stats(tb, variation)
                for x, y in stats.items():
                    if x > 18:
                        q = pd.DataFrame(y)
                        print(q.to_string())


TF: 10m, ATR: 11, MULT: 1.5
        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio                    Variation
2019     173.9766          848  51.8868                1.0011             -0.6532      -8.5355      20.3828  TF: 10m, ATR: 11, MULT: 1.5
2020     249.4165          879  50.9670                1.7134             -1.2023     -34.6099       7.2065  TF: 10m, ATR: 11, MULT: 1.5
2021     110.8479          949  47.3130                1.0935             -0.7603     -13.2280       8.3798  TF: 10m, ATR: 11, MULT: 1.5
2022      97.0716          855  50.1754                1.0551             -0.8347     -16.6586       5.8271  TF: 10m, ATR: 11, MULT: 1.5
2023      16.6786          887  44.6449                0.7304             -0.5551     -19.0339       0.8763  TF: 10m, ATR: 11, MULT: 1.5
2024      82.2507         1049  47.2831                0.8945             -0.6535     -16.9595       4.8498  TF: 10m, ATR: 11, MULT: 1.5
2025      26.